<a href="https://colab.research.google.com/github/saeedt70/persian-text-classification-/blob/main/lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openpyxl
!pip install hazm

In [ ]:
import numpy as np 
import pandas as pd
from hazm import *
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import seaborn as sns
import os
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/Saeed taheri.xlsx')
df.shape

In [ ]:
!pip install wordcloud-fa -qqq
!pip install gradio -qqq
import pandas as pd # to manipulate the dataset
from wordcloud_fa import WordCloudFa # to create wordcloud out of persian text
import gradio as gr # to build & share delightful apps

from collections import Counter
text1 = values = ''.join(str(v) for v in df['text'])

wc = WordCloudFa( background_color="white", width=1000, height=500)
word_cloud = wc.generate(text1)
image = word_cloud.to_image()
image

##Data set Plot

In [ ]:
isni=df.isnull().sum()
print(isni)
sns.countplot(x=df["cat"])
plt.show()

sns.countplot(x=df["sent"])
plt.show()
#isnull



In [ ]:
import re
stop=stopwords_list()
normalizer = Normalizer()
df = df.dropna()
df['text'] = df['text'].str.replace('[^\w\s#@/:%.,_-]', '', flags=re.UNICODE).str.lower()




In [ ]:
df['text'] = df['text'].apply(lambda x:normalizer.normalize(x))

df['text'] = df['text'].apply(lambda x: re.sub('[0-9]+', ' ', x))
df['text'] = df['text'].apply(lambda x: [item for item in x.split() if item not in stop])
To_Process = df[['cat','text']]
To_Process.shape

In [ ]:
train, test = train_test_split(To_Process, test_size=0.1, random_state=42, shuffle=True)



In [ ]:
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences


# Define the sequence lengths, max number of words and embedding dimensions
MAX_SEQUENCE_LENGTH = 300 #can plot a graph for length
MAX_NB_WORDS = 15000
EMBEDDING_DIM = 100

# Get the frequently occurring words
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(train.text)
train_sequences = tokenizer.texts_to_sequences(train.text)
test_sequences = tokenizer.texts_to_sequences(test.text)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

#some padding shit
train_data = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [ ]:
from sklearn.preprocessing import LabelEncoder

train_labels = train['cat']
test_labels = test['cat']

le = LabelEncoder()
le.fit(train_labels)
train_labels = le.transform(train_labels)
test_labels = le.transform(test_labels)
print(le.classes_)

In [ ]:

from tensorflow.keras.utils import to_categorical

#change data types (actualy idk why)
labels_train = to_categorical(np.asarray(train_labels))
labels_test = to_categorical(np.asarray(test_labels))

In [ ]:
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.models import Model
from keras.layers import Dropout, Input
from keras.layers import Bidirectional
from keras.layers import Activation, Dense,GRU,Flatten,LSTM
from keras.layers import Embedding

model2 = Sequential()
model2.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model2.add(Bidirectional(LSTM(64,  activation='relu',  dropout=0.2,recurrent_regularizer='l1',return_sequences=True)))
#model2.add(GRU(128, recurrent_regularizer='l1',return_sequences=True))
#model2.add(Bidirectional(GRU(128)))
#model2.add(LSTM(64,  activation='relu',recurrent_regularizer='l1',return_sequences=True))
model2.add(Dropout(0.2))
model2.add(BatchNormalization())
model2.add(Flatten())
model2.add(Dense(9,activation='softmax'))
model2.compile(loss = 'categorical_crossentropy',optimizer='adam',metrics = ['accuracy'])
print(model2.summary())

In [ ]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True)



In [ ]:
histories = {'accuracy':[], 'loss':[], 'val_accuracy':[], 'val_loss':[]}
for train_idx, val_idx in kfold.split(train_data):
    x_train_f = train_data[train_idx]
    y_train_f = labels_train[train_idx]
    x_val_f = train_data[val_idx]
    y_val_f = labels_train[val_idx] 

    history=model2.fit(x_train_f, y_train_f, batch_size=64,validation_data=(x_val_f, y_val_f), epochs=10,verbose = 1)
    histories['accuracy'].append(history.history['accuracy'])
    histories['loss'].append(history.history['loss'])
    histories['val_accuracy'].append(history.history['val_accuracy'])
    histories['val_loss'].append(history.history['val_loss'])

In [ ]:

model2.save('bigru0')

#plot

In [ ]:
accr= model2.evaluate(test_data,labels_test)
print(accr)
print('accuracy')
print(histories['accuracy'])
print('loss')
print(histories['loss'])
print('val_accuracy')
print(histories['val_accuracy'])
print('val_loss')
print(histories['val_loss'])



from matplotlib import pyplot as plt
plt.plot( histories['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('fold')
#plt.legend(['train', 'val'], loc='upper left')
plt.show()

###########los
plt.plot( histories['loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('fold')
#plt.legend(['train', 'val'], loc='upper left')
plt.show()
################################

plt.plot(histories['val_loss'])
plt.title('val_losss')
plt.ylabel('loss')
plt.xlabel('fold')
#plt.legend(['accuracy', 'loss'], loc='upper left')
plt.show()
################################
plt.plot(histories['val_accuracy'])
plt.title(' val-accuracy ')
plt.ylabel('accuracy')
plt.xlabel('fold')
#plt.legend(['val-accuracy', 'val-loss'], loc='upper left')
plt.show()

In [ ]:
import sklearn
from sklearn.metrics import precision_recall_fscore_support as score

predicted=model2.predict(test_data)
predicted

precision, recall, fscore, support = score(labels_test, predicted.round())
print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))
print("############################")
print(sklearn.metrics.classification_report(labels_test, predicted.round()))

###############################


In [ ]:
import sklearn
from IPython import display
import matplotlib.pyplot as plt

target =  ['ghimat', 'internet', 'fun', 'hard','bag','time&level','update','other','spam']
#target =  ['manfi', 'khonsa', 'mosbat'] 
report1 =sklearn.metrics.classification_report(labels_test, predicted.round(),target_names=target, digits=4,  output_dict=True)
display.display(pd.DataFrame(report1))
df = pd.DataFrame(report1)
df.iloc[:3, :9].T.plot(kind='bar')
plt.show()

In [ ]:
predicted

In [ ]:
k = 1024
print(test.iloc[k])
x = model2(test_data[k:k+1,:])
x


In [ ]:
from sklearn.metrics import *
import matplotlib.pyplot as plt
import seaborn as sns


y_pred = np.argmax(predicted.round(), axis=1)
cm=multilabel_confusion_matrix(test_labels, y_pred, labels=[1, 2, 3,4,5,6,7,8,9] )
print(cm)

#############
# Plot confusion matrix 
fig = plt.figure(figsize = (15, 10))
#['manfi', 'khonsa', 'mosbat']
#['ghimat', 'internet', 'fun', 'hard','bag','time&level','update','other','spam']
for i, (label, matrix) in enumerate(zip(['ghimat', 'internet', 'fun', 'hard','bag','time&level','update','other','spam'], cm)):
    plt.subplot(f'33{i+1}')
    labels = [f'{label}', label]
    sns.heatmap(matrix, annot = True, square = True, fmt = 'd', cbar = False, cmap = 'Blues', 
                xticklabels = labels, yticklabels = labels, linecolor = 'black', linewidth = 1)
    plt.title(labels[0])

plt.tight_layout()
plt.show()
